## AutoEncoders on PointClouds - Modular Code

In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.point_net_ae import Configuration as PN_Conf

# import tf_lab.point_clouds.various_encoders_decoders as enc_dec
# import tf_lab.models.point_net_based_AE as pnAE

from general_tools.in_out import create_dir
from geo_tool import Point_Cloud

In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

seed = None
# seed = 42
# np.random.seed(seed)
# tf.set_random_seed(seed)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
data_path = '/Users/optas/DATA/Point_Clouds/Shape_Net_Core/from_manifold_meshes/1024/03001627'
train_dir = '/Users/optas/DATA/Neural_Nets/Models/Point_Cloud_AE/'

experiment_name = 'scratch'
train_dir = osp.join(train_dir, experiment_name)

create_dir(train_dir)

'/Users/optas/DATA/Neural_Nets/Models/Point_Cloud_AE/scratch'

In [7]:
file_names = pio.load_filenames_of_input_data(data_path)
file_names = file_names[:1000]

pclouds, model_names = pio.load_crude_point_clouds(file_names=file_names, n_threads=11)




train_data_, val_data_, test_data_ = pio.train_validate_test_split([pclouds, model_names],
                                                                   train_perc=0.8,
                                                                   validate_perc=0.1,
                                                                   test_perc=0.1,
                                                                   seed=seed)

train_data = PointCloudDataSet(train_data_[0], labels=train_data_[1])
val_data = PointCloudDataSet(val_data_[0], labels=val_data_[1])
test_data = PointCloudDataSet(test_data_[0], labels=test_data_[1])

# train_data = PointCloudDataSet(train_data_[0], noise={'frac':0.20, 'filler':0.0})
# val_data = PointCloudDataSet(val_data_[0], noise={'frac':0.20, 'filler':0.0})
# test_data = PointCloudDataSet(test_data_[0], noise={'frac':0.20, 'filler':0.0})

6778 files containing  point clouds were found.


In [13]:
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

conf = PN_Conf(n_input = [1024, 3],
           training_epochs = 16,
           batch_size = 20,
           loss = 'l2',
           train_dir = train_dir,
           loss_display_step = 1,               
           saver_step = 2,
           learning_rate = 0.0002,
          )

In [18]:
reset_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
ae.train_names = set(train_data_[1])
ae.train(train_data, conf)

In [22]:
reset_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
ae.restore_model(conf.train_dir, 10)

Model restored in epoch 10.


In [52]:
a = test_data.point_clouds
b = val_data.point_clouds
a = np.vstack((a, b))

_, new_val_data_, new_test_data_ = pio.train_validate_test_split([a],
                                                                   train_perc=0,
                                                                   validate_perc=0.5,
                                                                   test_perc=0.5,)


new_val_data = PointCloudDataSet(new_val_data_)
new_test_data = PointCloudDataSet(new_test_data_)

conf.debug = False
_, loss_tes, _ = ae.evaluate(new_test_data, conf)
_, loss_val, _ = ae.evaluate(new_val_data, conf)
if loss_val < loss_tes:
    print 'val winner'
else:
    print 'test winner'

val winner


In [37]:
from tf_lab.fundamentals.inspect import hist_summary_of_trainable
writer = tf.summary.FileWriter('/Users/optas/Desktop/test', ae.graph)
# hist_summary_of_trainable(ae.graph)